# Time Series 

### Basic

시간에 따른 일련의 데이터를 Time series data라고 합니다. <br>
단 한개의 변수로 이루어진 time series를 **univariate** 이라고 하며, 2개 이상의 변수로 이루어진 time series를 **multivariate**이라고 합니다.


### Trend, Cyclical, Seasonal, Irregular 

Time series는 일반적으로 4개의 요소 (Trend, Cyclical, Seasonal 그리고 Irregular)에 의해 영향을 받습니다.

1. **Trend**: 증가하는지, 감소하는지, 또는 긴 시간동안 정체되어 있는지등의 추세를 Trend라고 합니다.<br>Trend는 time series안에서 long term movement를 보는 것이 중요합니다.
2. ****


# AR (Autoregressive) Model

AR(p) (AutoRegressive of order p)는 간단하게 과거의 패턴이 지속된다면 <br>
시계열 데이터 관측치 $ y_t $ 는 과거 관측치 $ y_{t-1} $, $ y_{t-2} $, $ y_{t-3} $, ... 에 의해 예측될 수 있다고 봅니다. <br>
오직 과거의 데이터만 사용해서 예측 모형을 만들게 되며, 이름도 autoregressive 입니다. (그리스어로 auto 라는 뜻은 "self"를 가르킵니다.)<br>



$$ X_t = \alpha_1 X_{t-1} + ... + \alpha_p X_{t-p} + \epsilon_t $$ 

* **order:** $ p $
* **parameters (coefficients):** $ \alpha_1 $, $ \alpha_2 $, ..., $ \alpha_p $
* **error term (white noise):** $ \epsilon_t $

# Auto-Regressive Integrated Moving Average (ARIMA)

ARIMA를 적용하기 위해서는 time series는 linear 하며 통계적 분포(normal distribution같은 statistical distribution)을 따른다고 가정합니다. <br>
다음의 모델들의 특징을 모두 갖고 있습니다.

* Autoregressive (AR)
* Moving Average (MA)

# References 

* http://users.dma.unipi.it/~flandoli/AUTCap4.pdf
* https://gejza.nipax.cz/_media/stochasticke_procesy:1302.6613.pdf
* https://onlinecourses.science.psu.edu/stat510/